In [ ]:
#IMPLEMENTING NAIVE BAYES AND RANDOM FOREST ALGOS

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action = "ignore")
gc = pd.read_csv("C:\\Users\\Vaibhav\\Desktop\\BA\\Datasets\\German_credit_classification.csv")
#Very famous German Credit Dataset 
#Whether a person will default on the credit or not - y variable is Default
#Step1. Data Exploration and Data Visualization
df = pd.DataFrame(gc)

In [2]:
gc.head(3)

,Default,checkingstatus1,duration,history,purpose,amount,savings,employ,installment,status,...,residence,property,age,otherplans,housing,cards,job,liable,tele,foreign
0,0,A11,6,A34,A43,1169,A65,A75,4,A93,...,4,A121,67,A143,A152,2,A173,1,A192,A201
1,1,A12,48,A32,A43,5951,A61,A73,2,A92,...,2,A121,22,A143,A152,1,A173,1,A191,A201
2,0,A14,12,A34,A46,2096,A61,A74,2,A93,...,3,A121,49,A143,A152,1,A172,2,A191,A201


---

### Creating dummy variables using pd.get_dummies

In [3]:
table = pd.crosstab(gc.checkingstatus1, gc.Default)
table = np.array(table)
pd.DataFrame(table/np.sum(table, axis = 1).reshape(-1,1), columns= ["Non-Default_Rate", "Default_Rate"])

,Non-Default_Rate,Default_Rate
0,0.507299,0.492701
1,0.609665,0.390335
2,0.777778,0.222222
3,0.883249,0.116751


In [4]:
#check for all the categorical variables

In [5]:
cat_vars = ['checkingstatus1', 'history', 'purpose', 'savings', 'employ', 'status', \
            'others', 'property', 'otherplans', 'housing','job','tele','foreign']

In [6]:
#Create dummies for the categorical columns
for cat in cat_vars:
    dummy = pd.get_dummies(gc[cat], prefix = cat+'_',drop_first=True)
    gc = pd.concat([gc,dummy], axis = 1)
print(gc.columns)
print(gc.shape)
gc.to_csv("dummy_German.csv")

Index(['Default', 'checkingstatus1', 'duration', 'history', 'purpose',
       'amount', 'savings', 'employ', 'installment', 'status', 'others',
       'residence', 'property', 'age', 'otherplans', 'housing', 'cards', 'job',
       'liable', 'tele', 'foreign', 'checkingstatus1__A12',
       'checkingstatus1__A13', 'checkingstatus1__A14', 'history__A31',
       'history__A32', 'history__A33', 'history__A34', 'purpose__A41',
       'purpose__A410', 'purpose__A42', 'purpose__A43', 'purpose__A44',
       'purpose__A45', 'purpose__A46', 'purpose__A48', 'purpose__A49',
       'savings__A62', 'savings__A63', 'savings__A64', 'savings__A65',
       'employ__A72', 'employ__A73', 'employ__A74', 'employ__A75',
       'status__A92', 'status__A93', 'status__A94', 'others__A102',
       'others__A103', 'property__A122', 'property__A123', 'property__A124',
       'otherplans__A142', 'otherplans__A143', 'housing__A152',
       'housing__A153', 'job__A172', 'job__A173', 'job__A174', 'tele__A192',
       

In [7]:
gc.drop(cat_vars, axis = 1, inplace=True)
gc.shape

(1000, 49)

---

In [8]:
from sklearn.model_selection import train_test_split
train , test = train_test_split(gc, test_size = 0.30, random_state = 2) 
X_train = train.drop(['Default'], axis = 1)
y_train = train['Default']

X_test = test.drop(['Default'], axis = 1)
y_test = test['Default']
X_train.head(2)

,duration,amount,installment,residence,age,cards,liable,checkingstatus1__A12,checkingstatus1__A13,checkingstatus1__A14,...,property__A124,otherplans__A142,otherplans__A143,housing__A152,housing__A153,job__A172,job__A173,job__A174,tele__A192,foreign__A202
90,12,618,4,4,56,1,1,0,0,1,...,0,0,1,1,0,0,1,0,0,0
305,6,1543,4,2,33,1,1,0,0,1,...,0,0,1,1,0,0,1,0,0,0


In [9]:
X_test.head(2)

,duration,amount,installment,residence,age,cards,liable,checkingstatus1__A12,checkingstatus1__A13,checkingstatus1__A14,...,property__A124,otherplans__A142,otherplans__A143,housing__A152,housing__A153,job__A172,job__A173,job__A174,tele__A192,foreign__A202
37,18,2100,4,2,37,1,1,0,1,0,...,0,1,0,1,0,0,1,0,0,0
726,15,1316,2,2,47,2,1,0,0,1,...,0,0,1,1,0,1,0,0,0,0


In [10]:
train.head(1)

,Default,duration,amount,installment,residence,age,cards,liable,checkingstatus1__A12,checkingstatus1__A13,...,property__A124,otherplans__A142,otherplans__A143,housing__A152,housing__A153,job__A172,job__A173,job__A174,tele__A192,foreign__A202
90,0,12,618,4,4,56,1,1,0,0,...,0,0,1,1,0,0,1,0,0,0


In [11]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
sc.fit(X_train, y_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)
X_train = pd.DataFrame(X_train, columns = train.columns[1:])
X_test = pd.DataFrame(X_test, columns = test.columns[1:])

In [12]:
print(X_train.head(1))
print(X_test.head(1))

   duration    amount  installment  residence       age  cards  liable  \
0  0.142857  0.020249          1.0        1.0  0.660714    0.0     0.0   

   checkingstatus1__A12  checkingstatus1__A13  checkingstatus1__A14  ...  \
0                   0.0                   0.0                   1.0  ...   

   property__A124  otherplans__A142  otherplans__A143  housing__A152  \
0             0.0               0.0               1.0            1.0   

   housing__A153  job__A172  job__A173  job__A174  tele__A192  foreign__A202  
0            0.0        0.0        1.0        0.0         0.0            0.0  

[1 rows x 48 columns]
   duration    amount  installment  residence       age  cards  liable  \
0      0.25  0.101794          1.0   0.333333  0.321429    0.0     0.0   

   checkingstatus1__A12  checkingstatus1__A13  checkingstatus1__A14  ...  \
0                   0.0                   1.0                   0.0  ...   

   property__A124  otherplans__A142  otherplans__A143  housing__A152  

### NAIVE BAYES CLASSIFIER
- Based on conditional probability as explained in the excel sheet. 
- GaussianNB - makes the assumption that continuous features follow Gaussian distribution / Normal Distribution
- Multinomial NB - makes the assumption that the features follow multinomial distribution (1,2,3,4,5,...)
- Bernoulli NB - makes the assumption that the features follow bernoulli dist 0 and 1

NB algo is a parametric algo: which basically means that it makes strong assumptions about the distribution shape (Gaussian shape) and about the independence of the features. 

Naive Bayes' is a Naive algo because it makes strong assumptions about the independence of the features, which in real life is never true. 

Unlike KNN(lazy learner) is an eager learning algorithm. NB is used when the number of dimensions / features is very high eg. in NLP Problems 

It has got applications in weather forecasting like prediction of rainfall based on features like humidity, temperature etc.
- It's used in medical diagnosis. Given a list of symptoms, predict the prob. of desease
- Spam and ham classification of emails - gmail

In [13]:
from sklearn.naive_bayes import GaussianNB
#MultinomialNB for multinomial classification problem
gb = GaussianNB()

mod = gb.fit(X_train,y_train)

ypred = mod.predict(X_test)
ypred

array([0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1], d

In [14]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,ypred)

0.72

In [15]:
from sklearn.model_selection import cross_val_score
accuracy = cross_val_score(mod,X_train,y_train, scoring='accuracy', cv = 10)
print(accuracy)
print("Accuracy of Naive Bayes is: " , accuracy.mean())

[0.73239437 0.4        0.75714286 0.61428571 0.67142857 0.74285714
 0.67142857 0.71428571 0.75714286 0.62318841]
Accuracy of Naive Bayes is:  0.6684154200565713


---

### RANDOM FORESTS
RANDOM FORESTS
It’s an ensemble technique – Each model is a weak learner in itself. But when we combine multiple weak learners, we get a strong learner. 
Bagging – Bootstrapped aggregation of data. The algo will take multiple samples with replacement of size 2/3rd of the training data-set. The model will create a decision tree (or any other weak learner model) on each sample and make predictions on the out of bag data. This way we can create multiple models (500, 1000, 2000 until the learning is improving). Ultimately we are going to make the final predictions using aggregation (i.e voting of y values incase of classification and average of values in case of regression). We are reducing the standard- deviation  based on CLT – when we take the sampling distribution of size n, the standard deviation will reduce to sigma/(n^.5). This will reduce the bias error on the training data.
Random Forest model is applied above bagging. The algo will select x number of IV randomly and create the decision tree (or any other weak learner model) and the bagging technique will be applied i.e the predictions will be finalized based on aggregation, as discussed earlier. Gen x = p^.5. No. of features is a hyper-parameter which needs be tuned. By doing this we are trying to de-correlate our trees.  We are reducing the multi-collinearity in our model, which is helping us reduce the over-fitting. As we increase the number of trees, we are minimizing our bias error (which can lead to over-fitting)
We will also reduce the depth of our weak learner models to avoid over-fitting on the data. This means that each of the 2000 trees constructed in the randomforest model are going to communicate the most important split for them. Ultimately aggregation will be done based on voting (classification)/ average (regression).


In [51]:
from sklearn.ensemble import RandomForestClassifier
rf_class = RandomForestClassifier(n_estimators=10, random_state=2)

In [52]:
rf_class.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=2, verbose=0, warm_start=False)

In [53]:
ypred = rf_class.predict(X_test)

In [54]:
accuracy_score(y_test, ypred)

0.72

In [55]:
from sklearn.model_selection import cross_val_score
accuracy = cross_val_score(rf_class,X_train,y_train, scoring='accuracy', cv = 10)
print(accuracy)
print("Accuracy of Random Forests is: " , accuracy.mean())

[0.71830986 0.74285714 0.72857143 0.72857143 0.74285714 0.78571429
 0.68571429 0.67142857 0.72857143 0.7826087 ]
Accuracy of Random Forests is:  0.7315204269092818


In [56]:
rf_class.get_params

<bound method BaseEstimator.get_params of RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=2, verbose=0, warm_start=False)>

In [85]:
params = {
             'max_depth': [3,4,10],#5],#,6,7],
             'n_estimators': [250, 500]#,750,1000],#, 1000],
             #'max_features': ['auto']# 'log2', 'sqrt']
}

In [86]:
from sklearn.grid_search import GridSearchCV
grid = GridSearchCV(estimator = rf_class, param_grid=params,scoring='f1', cv=5, n_jobs=-1)

In [87]:
grid.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=2, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': [3, 4, 10], 'n_estimators': [500, 750, 1000]},
       pre_dispatch='2*n_jobs', refit=True, scoring='f1', verbose=0)

In [88]:
grid.best_params_

{'max_depth': 10, 'n_estimators': 1000}

In [89]:
ypred = grid.predict(X_test)

In [90]:
import pandas as pd
pd.crosstab(y_test, ypred)

col_0,0,1
Default,,
0,190,11
1,72,27


In [91]:
accuracy_score(y_test,ypred)

0.7233333333333334

In [92]:
from sklearn.metrics import f1_score
f1_score(test.Default,ypred)

0.3941605839416058